In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import re

In [2]:
fake = pd.read_csv("data/raw/Fake.csv")
real = pd.read_csv("data/raw/true.csv")

In [3]:
# np.set_printoptions(threshold=np.inf)
# arr = true["date"].to_numpy()
# #print(arr)

In [4]:
fake['label'] = 'fake'
real['label'] = 'real'

In [5]:
pat1 = r'^[A-Z]+'
pat2 = r'(/)[A-Z]+'
pat3 = r'\(Reuters\)\s[-]\s'
combined_pat = r'|'.join((pat1, pat2, pat3))

In [6]:
real['text'] = real['text'].str.replace(combined_pat, '', regex=True)
real['text']

0         The head of a conservative Republican faction...
1         Transgender people will be allowed for the fi...
2         The special counsel investigation of links be...
3         Trump campaign adviser George Papadopoulos to...
4         President Donald Trump called on the U.S. Pos...
                               ...                        
21412     NATO allies on Tuesday welcomed President Don...
21413     LexisNexis, a provider of legal, regulatory a...
21414     In the shadow of disused Soviet-era factories...
21415     Vatican Secretary of State Cardinal Pietro Pa...
21416     Indonesia will buy 11 Sukhoi fighter jets wor...
Name: text, Length: 21417, dtype: object

In [7]:
all_news = pd.concat([real, fake])
all_news = all_news.replace(r'^\s*$', np.nan, regex=True)
all_news = all_news.dropna(subset=['text'])
all_news.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [8]:
match_condn = r'\b([0-9])\b'
replace_str = r'0\1'

all_news['date'] = all_news['date'].str.replace(match_condn, replace_str, regex=True)

In [9]:
all_news.date = all_news.date.str.strip()

In [10]:
replacement = {
    "^Jan\s": "January ",
    "^Feb\s": "February ",
    "^Mar\s": "March ",
    "^Apr\s": "April ",
    "^Jun\s": "June ",
    "^Jul\s": "July ",
    "^Aug\s": "August ",
    "^Sep\s": "September ",
    "^Oct\s": "October ",
    "^Nov\s": "November ",
    "^Dec\s": "December ",
    "14[-]Feb[-]18$": "February 14, 2018",
    "15[-]Feb[-]18$": "February 15, 2018",
    "16[-]Feb[-]18$": "February 16, 2018",
    "17[-]Feb[-]18$": "February 17, 2018",
    "18[-]Feb[-]18$": "February 18, 2018",
    "19[-]Feb[-]18$": "February 19, 2018",
}

all_news['date'] = all_news['date'].replace(replacement, regex=True)

In [11]:
# np.set_printoptions(threshold=np.inf)
# arr = all_news['date'].to_numpy()
# print(arr)

In [12]:
all_news['date'] = pd.to_datetime(all_news['date'], format='%B %d, %Y', errors='coerce')
all_news

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction...,politicsNews,2017-12-31,real
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fi...,politicsNews,2017-12-29,real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links be...,politicsNews,2017-12-31,real
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos to...,politicsNews,2017-12-30,real
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Pos...,politicsNews,2017-12-29,real
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,2016-01-16,fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,2016-01-16,fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,2016-01-15,fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,2016-01-14,fake


In [13]:
all_news.isna().sum()

title       0
text        0
subject     0
date       10
label       0
dtype: int64

In [14]:
# all_news.to_csv("data/processed/all_news.csv", index=False)

In [15]:
all_news.query("subject == 'left-news'")

,title,text,subject,date,label
17461,BARBRA STREISAND Gives Up On Dream Of Impeachi...,Barbra Streisand was an Obama sycophant and on...,left-news,2017-12-31,fake
17462,WATCH: SENATOR LINDSEY GRAHAM DROPS BOMBSHELL…...,Everyone suspected the sketchy Steele Dossier ...,left-news,2017-12-31,fake
17463,“CONSERVATIVE GAY GUY” BLASTS Pence’s Aspen Ne...,It s been said that good fences make good neig...,left-news,2017-12-30,fake
17464,WHITE COLLEGE SNOWFLAKES Can ‘Identify’ As Bla...,The latest video from Campus Reform shows just...,left-news,2017-12-30,fake
17465,BILL NYE The FAKE Science Guy THREATENS Conser...,"Friday on MSNBC, climate activist Bill Nye war...",left-news,2017-12-30,fake
...,...,...,...,...,...
21915,ENTITLED IRS ETHICS LAWYER DISBARRED FOR ETHIC...,Don t you just love an entitled IRS lawyer who...,left-news,2015-04-04,fake
21916,[VIDEO] 16 YR OLD ARRESTED For Violent Gang Be...,This is a sad commentary on a generation who h...,left-news,2015-04-04,fake
21917,“Non-violence hasn’t worked”…Reverend Sam Most...,Yeah that whole taking up arms thing seems t...,left-news,2015-04-01,fake
21918,WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY...,"In case you missed it Sen. Harry Reid (R-NV), ...",left-news,2015-03-31,fake


In [16]:
# Limiting categories of Subjects to only 2 (politicsNews and worldnews)
subject_replace = {'News':'other', 
                   'politics':'politicsNews', 
                   'left-news' : 'other',
                   'Government News': 'politicsNews',
                   'US_News': 'politicsNews',
                   'Middle-east': 'worldnews' 
            }

all_news = all_news.replace({"subject": subject_replace})

#Test df to check subjects have been replaced
all_news.query("subject == 'politics'")

,title,text,subject,date,label


In [17]:
all_news.to_csv("data/processed/all_news.csv", index=False)